**[Dataset Download 1](https://s3.amazonaws.com/bebi103.caltech.edu/data/gardner_time_to_catastrophe_dic_tidy.csv)** <br/>
**[Dataset Download 2](https://s3.amazonaws.com/bebi103.caltech.edu/data/gardner_mt_catastrophe_only_tubulin.csv)**<br/>
Put it in a `data/` directory.

In [1]:
import microtubule_analysis as ma
import pandas as pd

import bebi103
import bokeh_catplot
import bokeh.io
import numpy as np

bokeh.io.output_notebook()

Loading BokehJS ...

# Fitting a Distribution to Microtubule Catastrophe 

This analysis follows data gathered by 
[Garner, Zanic, and coworkers](https://doi.org/10.1016/j.cell.2011.10.037). In their experiment, they find and analyze catastrophe times for labeled and unlabeled microtubules. They also gather catastrophe times for differing concentrations of tubulin. The data used in our analysis can be downloaded from **[here](https://s3.amazonaws.com/bebi103.caltech.edu/data/gardner_time_to_catastrophe_dic_tidy.csv)** and
**[here](https://s3.amazonaws.com/bebi103.caltech.edu/data/gardner_mt_catastrophe_only_tubulin.csv)**. Put it in a `data/` folder.

In our analysis, we will first find if labeling the microtubules affects catastrophe times. Then, we will choose a couple of possible distributions for catastrophe times. Finally, we use maximum likelihood estimation to estimate parameters for the different distributions, and we choose a distribution to model microtubule catastrophe.

## Does labeling microtubules affect catastrophe times? 

First, we must find out if the labeled and unlabeled mucrotubules have about the same times to catastrophe.

In [2]:
df_labeled_unlabeled = pd.read_csv('data/gardner_time_to_catastrophe_dic_tidy.csv')
df_labeled_unlabeled = df_labeled_unlabeled.replace(True, 'Labeled')
df_labeled_unlabeled = df_labeled_unlabeled.replace(False, 'Unlabeled')


p = ma.plot_ecdf(
    tidy_data=df_labeled_unlabeled,
    cats='labeled',
    val='time to catastrophe (s)',
    title='ECDF of unlabeled and labeled tubulin',
    conf_int=True,
    width=400,
)

bokeh.io.show(p)

Based off of the confidence intervals on the ECDF plot, we can conclude that labeling the microtubules does not affect catastrophe times.

### Other Confidence Intervals 

To further ensure the labeled and unlabeled distributions are the same, we will derive other confidence intervals.

First, we will numerically generate the confidence interval using bootstrapping. To compare the two distributions, we will look at the mean.

In [3]:
labeled = df_labeled_unlabeled[
    df_labeled_unlabeled.labeled == 'Labeled'
]['time to catastrophe (s)'].values 

unlabeled = df_labeled_unlabeled[
    df_labeled_unlabeled.labeled == 'Unlabeled'
]['time to catastrophe (s)'].values 

labeled_ci = ma.generate_bs_confidence_interval(labeled, np.mean)
unlabeled_ci = ma.generate_bs_confidence_interval(unlabeled, np.mean)

ma.print_confidence_interval(*labeled_ci, title='Labeled')
ma.print_confidence_interval(*unlabeled_ci, title='Unlabeled')


Labeled confidence interval (95%) is: [399.59716, 479.19491]
Unlabeled confidence interval (95%) is: [350.45000, 473.48816]


The labeled and unlabeled samples have similar confidence intervals (similar means), but the width of the unlabeled sample's confidence interval is higher; there is more variance in the estimate. This makes sense; it is harder to get an unlabeled sample accurately.

Now, we will test the hypothesis that the labeled and unlabeled samples follow a different distribution versus the null that they do not. We will choose the mean as the test statistic for this, as it models the total time to catastrophe well.

In [4]:
p_val = ma.find_p_value_permutation(labeled, unlabeled)
print('The p-value is:', p_val)

The p-value is: 0.571


The p-value is nowhere near the level of rejection. Thus, we do not reject the null that the labeled and unlabeled samples follow the same distribution.

Next, we will find the labeled and unlabeled CIs using the normal distribution.

In [5]:
labeled_ci_normal = ma.ci_using_normal(labeled)
unlabeled_ci_normal = ma.ci_using_normal(unlabeled)

ma.print_confidence_interval(*labeled_ci_normal, 'Labeled Normal')
ma.print_confidence_interval(*unlabeled_ci_normal, 'Unlabeled Normal')

Labeled Normal confidence interval (95%) is: [400.83602, 480.58578]
Unlabeled Normal confidence interval (95%) is: [351.16508, 473.88755]


Both of these confidence intervals were very similar to those that we obtained by bootstrapping. This further verifies those results.

Lastly, we will make confidence intervals using DKW bounds and plot those ECDFs.

In [6]:
p = ma.plot_ecdf_dkw(unlabeled, 'Unlabeled')
bokeh.io.show(p)

In [7]:
p = ma.plot_ecdf_dkw(labeled, 'Labeled', color='blue')
bokeh.io.show(p)

In [8]:
p = ma.plot_ecdf_dkw(unlabeled, 'Unlabeled', p=p, color='orange')
bokeh.io.show(p)

These confidence intervals are much larger. They overlap quite well though.

All of the confidence intervals above imply that labeling the microtubules does not significantly affect the time to catastrophe.

## Possible distributions for microtubule catastrophe

We are going to assume that in order for microtubule catastrophe to occur, $n$ independent events must happen in order. Since these events are not that often, we can infer that they are Poisson Processes. Then, there are two possible distributions that we will discuss for microtubule catastrophe.

### The Gamma Distribution 

Microtubule catastrophe is a multistep process. If each step is modeled as an identical Poisson Process, then the time for each step, $X_i$, to happen would follow an exponential distribution with rate $\lambda$. The story for the Gamma distribution states that if we have iid steps $X_1, ..., X_{\alpha}$ that each take place in i.i.d. exponential time, then the total time between events, $X_1 + ... + X_\alpha$,
follows a gamma distribution, if the rate between each process is assumed to be the same.

### The 2 Poisson Process Distribution 

Another possible distribution that we will model is the "2 Poisson Process Distribution." This consists of two independent Poisson Processes happening after one another. These Poisson Processes can have different rates, unlike the Gamma Distribution, but there can only be 2 events.

Next, we will generate random samples of this and plot the ECDFs, to see how it looks.

In [9]:
beta_1 = 1
beta_2_values = [.125, .25, .5, 1, 2, 4, 8]

p = ma.create_generated_ecdfs(beta_1, beta_2_values, n_samples=5000)
bokeh.io.show(p)

As $\beta_2$ increases relative to $\beta_1$, the ECDF gets much steeper. This makes sense; the second event happens faster.

Now, knowing that the cumulative density function for this distribution is 
\begin{align}
F(t; \beta_1, \beta_2) = 
\frac{\beta_1 \beta_2}{\beta_2-\beta_1}\left[
\frac{1}{\beta_1}\left(1-\mathrm{e}^{- \beta_1 t}\right)- \frac{1}{\beta_2}\left(1-\mathrm{e}^{-\beta_2 t}\right)
\right],
\end{align}
we will define the probability density function.

To show this analytically, we will use the fact that if the first event happens in time $t_1$, then in order for the total time to equal $t$, the second event must happen in time $t - t_1$. We can then say that $f(t | t_1) = f_{e2}(t - t_1)$. Then, 
we can find the joint distribution $f(t, t_1) = f(t | t_1) * f(t_1) = f_{e1}(t_1) f_{e2}(t - t_1) = 
\beta_1 \exp(-\beta_1 t_1) * \beta_2 \exp(-\beta_2 (t - t_1) = \beta_1 \beta_2 \exp(t_1 (\beta_2 - \beta_1) - \beta_2 t)$. 

We know that $t_1$ will go from 0 to $t$. Using this, we can find the marginal distribution, $f(t)$. 
\begin{align}
    f(t) &= \int_0^t f(t, t_1) d t_1 \\
    &= \int_0^t \beta_1 \beta_2 \exp(t_1 (\beta_2 - \beta_1) - \beta_2 t) d t_1
\end{align}

We set $u = t_1 (\beta_2 - \beta_1) - \beta_2 t$, $du = (\beta_2 - \beta_1) dt_1$ and substitute it in.
\begin{align}
    f(t) &= \frac{\beta_1 \beta_2}{\beta_2 - \beta_1} \int \exp(u) du \\
    &= \frac{\beta_1 \beta_2}{\beta_2 - \beta_1} \left(\exp(t_1 (\beta_2 - \beta_1) - \beta_2 t)\right|^t_0 \\
    &= \frac{\beta_1 \beta_2}{\beta_2 - \beta_1} (\exp(t (\beta_2 - \beta_1) - \beta_2 t) - \exp(-\beta_2 t)) \\
    &= \frac{\beta_1 \beta_2}{\beta_2 - \beta_1} \left(e^{-\beta_1 t} - e^{-\beta_2 t}\right)
\end{align}

Thus, we have shown analytically that the PDF matching this story is $f(t;\beta_1, \beta_2) = \frac{\beta_1 \beta_2}{\beta_2 - \beta_1}\left(\mathrm{e}^{-\beta_1 t} - \mathrm{e}^{-\beta_2 t}\right)$. Now, we will show that an ECDF from our simulation matches the analytical CDF. We will choose $\beta_1 = 1, \beta_2 = 0.5$. 

In [10]:
p = ma.theo_empirical_cdf(beta_1=1, beta_2=0.5, n_samples=5000)
bokeh.io.show(p)

As seen in this plot, the empirical ane theoretical CDFs line up, showing that the original CDF is correct.

The two possible distributions that we will use to model microtubule catastrophe are the Gamma Distribution and the 2 Poisson Process Distribution.

##  MLE Estimation

Now, we will make use maximum likelihood estimation to estimate parameters for these two distributions. We use the labeled data, for we have shown that the labeling the microtubules does not significantly alter catastrophe time. Also, the labeled data has less variance, likely due to less measurement error.

### Gamma MLE 

First, we will estimate the parameters for the gamma distribution with this data set, and we will make a QQ plot to see how well it lines up.

In [11]:
alpha, beta = ma.mle_fun_gamma(labeled)
print('Alpha is: {}\nBeta is: {}'.format(alpha, beta))

Alpha is: 2.5211966515617266
Beta is: 0.00578934881663526


Now, we will generate a Q-Q plot using these estimated values of $\alpha$ and $\beta$.

In [12]:
p = bebi103.viz.qqplot(
    data=labeled,
    gen_fun=ma.generate_samples_gamma,
    args=(alpha, beta),
    n_samples=5000,
    x_axis_label="Time to Catastrophe (s)",
    y_axis_label="Time to Catastrophe (s)",
    title='Q-Q Plot for Gamma Distribution',
)

bokeh.io.show(p)

The Q-Q plot shows that the MLE estimates of $\alpha$ and $\beta$ line up quite well.

Now, we will generate the confidence intervals for $\alpha$ and $\beta$. we used the nonparametrized method; it is easy to draw samples from the data. The method itself draws `size` bootstrap samples from the data and fits an MLE estimator to each one. We then take the 2.5% and 97.5% percentiles. We will repeat this method for the 2 Poisson Process distribution.

In [13]:
bs_reps = ma.draw_bs_reps_mle(ma.mle_fun_gamma, labeled, size=5000, progress_bar=True)
conf_int = np.percentile(bs_reps, [2.5, 97.5], axis=0)
ma.print_confidence_interval(*conf_int[:, 0], 'alpha')
ma.print_confidence_interval(*conf_int[:, 1], 'beta')

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [01:26<00:00, 57.98it/s]


alpha confidence interval (95%) is: [2.07454, 2.79404]
beta confidence interval (95%) is: [0.00455, 0.00670]


With 95% certainty, we can say thta there are between 2.07303 and 2.81667 events leading to microtubule catastrophe, and these occur at an average rate between 0.00457 $s^{-1}$ and 0.00675 $s^{-1}$.

### 2 Poisson Process Distribution

First, we estimate the rates of the two processes using MLE.

In [14]:
beta_1, beta_2 = ma.mle_fun_2pp(labeled)
print('Beta 1 is: {}\nBeta 2 is: {}'.format(beta_1, beta_2))

Beta 1 is: 0.004041820126604453
Beta 2 is: 0.0049938339886770764


Once again, we will create a Q-Q plot using the estimated values

In [23]:
p = bebi103.viz.qqplot(
    data=labeled,
    gen_fun=ma.generate_samples_2pp,
    args=(beta_1, beta_2),
    n_samples=5000,
    x_axis_label="Time to Catastrophe (s)",
    y_axis_label="Time to Catastrophe (s)",
    title='Q-Q Plot for 2 Poisson Process Distribution',
)

bokeh.io.show(p)

This plot also fits the data very well. It hits off the mark a little bit in the beginning, compared to the Gamma Distribution, but it seems more accurate at later times.

Now, we will take the confidence intervals. A word of warning: It takes about 30 minutes to run a sample of size 1000.

In [16]:
bs_reps = ma.draw_bs_reps_mle(ma.mle_fun_2pp, labeled, size=1000, progress_bar=True)
conf_int = np.percentile(bs_reps, [2.5, 97.5], axis=0)
ma.print_confidence_interval(*conf_int[:, 0], 'beta 1')
ma.print_confidence_interval(*conf_int[:, 1], 'beta 2')

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [37:38<00:00,  2.26s/it]


beta 1 confidence interval (95%) is: [0.00342, 0.00481]
beta 2 confidence interval (95%) is: [0.00453, 0.00668]


Both events in this case are short; beta 1's is much less then the event rate in part a), and beta 2's rate is about the same as the rate in part a). This makes sense, though, because in part a), it was found that the number of events is greater than 2, according to the MLE. I would then expect that at least one of the rates would be smaller in part b), leading to an increase in total time taken. 

## Analysis of Microtubule Catastrophe 

Here, we use the second dataset, containing different concentrations of tubulin as they relate to catastrophe time. We will decide which distribution is better for modeling microtubule catastrophe, at least in this experiment.

First, we plot the time to catastrophe for each of the concentrations of tubulin as an ECDF.

### Choosing a Distribution

In [17]:
df_concentrations = pd.read_csv('data/gardner_mt_catastrophe_only_tubulin.csv', comment='#')
df_concentrations = ma.tidy_up_data(bad_df=df_concentrations)

p = ma.plot_ecdf(
    tidy_data=df_concentrations,
    cats='Tubulin Concentration (uM)',
    val='Time to Catastrophe (s)',
    title='Time to Catastrophe for Various Concentrations of Tubulin',
    width=450
)

bokeh.io.show(p)

In general, the time to catastrophe seems to increase as the tubulin concentration increases, but the 10.0 uM concentration overlaps with the 12.0 uM concentration. Similarly, the 9.0 uM concentration has shorter times to catastrophe towards the end than the 7.0 uM concentration.

Now, we will get the parameter estimates from both distributions at the 12 uM concentration.

In [18]:
twelve_um_data = ma.get_data_from_concentration(df_concentrations, 12)
alpha, beta = ma.mle_fun_gamma(twelve_um_data)
beta_1, beta_2 = ma.mle_fun_2pp(twelve_um_data)

print('For the Gamma Distribution:\nAlpha is: {}\nBeta is: {}\n'.format(
    alpha, 
    beta
))
print('For the 2 Poisson Distribution:\nBeta 1 is: {}\nBeta 2 is: {}'.format(
    beta_1, 
    beta_2
))

For the Gamma Distribution:
Alpha is: 2.9152818763887014
Beta is: 0.00766070314390043

For the 2 Poisson Distribution:
Beta 1 is: 0.0051263502481310765
Beta 2 is: 0.005296343019184135


In [19]:
ll_1 = ma.log_likelihood_function_gamma((alpha, beta), twelve_um_data)
ll_2 = ma.log_likelihood_function_2pp((beta_1, beta_2), twelve_um_data)

print('The AIC weight for the gamma distribution is {}.'.format(
    ma.get_aic_weight(ll_1, ll_2)
))

The AIC weight for the gamma distribution is 0.9999999999788296.


Based off of the AIC weights, we should definitely choose the Gamma distribution to model microtubule catastrophe. We will generate two Q-Q plots for good measure.

In [20]:
p = bebi103.viz.qqplot(
    data=twelve_um_data,
    gen_fun=ma.generate_samples_gamma,
    args=(alpha, beta),
    n_samples=5000,
    x_axis_label="Time to Catastrophe (s)",
    y_axis_label="Time to Catastrophe (s)",
    title='Q-Q Plot for Gamma Distribution',
)

bokeh.io.show(p)

In [21]:
p = bebi103.viz.qqplot(
    data=twelve_um_data,
    gen_fun=ma.generate_samples_2pp,
    args=(beta_1, beta_2),
    n_samples=5000,
    x_axis_label="Time to Catastrophe (s)",
    y_axis_label="Time to Catastrophe (s)",
    title='Q-Q Plot for 2 Poisson Process Distribution',
)

bokeh.io.show(p)

The two Q-Q plots indicate that we should choose the Gamma Distribution. It slightly deviates around 1500 s catastrophe times, but up to 1000 s, it models the data fairly well. For the 2 Poisson Process Distribution, it only models the data decently after 1000 s. Before that, it deviates a lot.

By performing these tests, we conclude the Gamma distribution models this data better than the 2 Poisson Process distribution.

### Getting Parameters for Every Concentration

Now we will get the $\alpha$ and $\beta$ parameters for each concentration. These will include confidence intervals.

In [22]:
df_new = ma.get_all_confidence_intervals_gamma(df_concentrations)
df_new

,Concentration,Alpha,Beta,CI_alpha,CI_beta
0,7.0,2.443909,0.007550,"[2.248747693634793, 2.688311309911024]","[0.006807247098605452, 0.008481359499871938]"
1,9.0,2.679864,0.008779,"[2.3440250760720964, 3.1352890253590986]","[0.007574152320812759, 0.010404948636501161]"
2,10.0,3.210823,0.009030,"[2.8139662076042464, 3.7716766964558053]","[0.00774257049462256, 0.01079298101273906]"
3,12.0,2.915282,0.007661,"[2.6049340922887056, 3.262475371808603]","[0.0067599257465602546, 0.008715814821584338]"
4,14.0,3.361506,0.007175,"[2.7419702729539375, 4.3906277443676185]","[0.005900932742501802, 0.009621718969611677]"


Based off of these parameters, we can say that with the highest concentrations, the rate of the events to occur is the smallest. The $\alpha$ parameters are fairly similar past a concentration of 9.0 um. However, the $\beta$ parameters decrease at 12.0 and 14.0 um, looking at the data. From this, we conclude that faster polymerization leads to a slower rate of catastrophe; the events that cause catastrophe to occur happen more slowly.